# Análisis Estadístico y Geoespacial de la Sismicidad en el Sureste de México (1975-2025)

**Autor:** Edgar Ivan Morales Herrera  
**Fecha:** Noviembre 2025

## 1. Resumen
El sureste de México es una región de alta complejidad tectónica donde interactúan las placas de Cocos, Norteamérica y el Caribe. Este proyecto analiza el catálogo sísmico de la región (USGS) para caracterizar patrones temporales y espaciales. El estudio se centra en distinguir entre tendencias sismológicas reales y sesgos introducidos por la evolución de la instrumentación científica.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygmt
from matplotlib.colors import ListedColormap

## 2. Carga y Limpieza de Datos
Se utilizan datos históricos desde 1950. Sin embargo, para garantizar la fiabilidad estadística, se aplica un filtro a partir de **1975** y magnitudes **M >= 3.5**, periodo en el cual la red de monitoreo global comenzó a tener una cobertura más consistente.

In [ ]:
# Carga de datos (Ajustar ruta si es necesario)
try:
    datos = pd.read_csv('./../datos/1950_2025.csv')
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Archivo no encontrado. Verifica la ruta del CSV.")

# Conversión de fechas
datos['time'] = pd.to_datetime(datos.time)
datos = datos.sort_values('time')

# FILTRO DE CALIDAD: 
# Nos quedamos con datos >= 1975 y Magnitud >= 3.5 para evitar sesgos de completitud
datos = datos[datos.time.dt.year >= 1975]
datos = datos[datos.mag >= 3.5]

# Variable de control para conteo
datos['control_ed'] = 1

print(f"Registros después del filtrado: {len(datos)}")

### 2.1 Ingeniería de Características (Feature Engineering)
Generamos columnas auxiliares para agrupar eventos por periodos (trimestres, cuatrimestres) y facilitar el análisis temporal.

In [ ]:
# Funciones auxiliares para agrupación temporal
def anyo_bim(x):
    mes = int(str(x)[5:7])
    if (mes>=1)&(mes<3): mes_ = '01'
    elif (mes>=3)&(mes<5): mes_ = '03'
    elif (mes>=5)&(mes<7): mes_ = '05'
    elif (mes>=7)&(mes<9): mes_ = '07'
    elif (mes>=9)&(mes<11): mes_ = '09'
    elif (mes>=11): mes_ = '11'
    return str(x)[:5] + mes_

def anyo_cuatri(x):
    mes = int(str(x)[5:7])
    if (mes>=1)&(mes<5): mes_ = '01'
    elif (mes>=5)&(mes<9): mes_ = '05'
    elif (mes>=9): mes_ = '09'
    return str(x)[:5] + mes_

datos['anyo_mes_bim'] = pd.to_datetime(datos.time.apply(anyo_bim))
datos['anyo_mes_cuatri'] = pd.to_datetime(datos.time.apply(anyo_cuatri))

# Agrupaciones para gráficas
datos_count_cuatri_mag = datos[['anyo_mes_cuatri','mag','control_ed']].groupby(by=['anyo_mes_cuatri','mag']).count().reset_index()

## 3. Análisis Temporal: ¿Está aumentando la sismicidad?

Al observar la gráfica de eventos por año, se nota una tendencia creciente. Es crucial interpretar esto con cuidado científico:

**Interpretación:** El incremento visible, especialmente en sismos de magnitud baja (< 5.0), **no implica necesariamente una mayor actividad geológica**, sino una **mejora en la capacidad de detección** (más estaciones, equipos digitales, mejor telemetría) por parte del Servicio Sismológico Nacional y agencias globales. La Tierra no tiembla más; nosotros escuchamos mejor.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
anyo = datos.time.dt.year.value_counts().sort_index().index
count = datos.time.dt.year.value_counts().sort_index().values

ax.bar(anyo, count, color='steelblue', alpha=0.8)
ax.set_ylabel('Cantidad de Eventos Detectados')
ax.set_title('Evolución del Registro Sísmico (1975-2025)')
ax.grid(axis='y', alpha=0.3)
plt.show()

### 3.1 Distribución Magnitud-Tiempo
El siguiente gráfico de dispersión muestra la relación entre la fecha y la magnitud. Note cómo la densidad de puntos pequeños aumenta con el tiempo (reflejo de la instrumentación), mientras que los grandes terremotos (puntos rojos/naranjas) ocurren con una frecuencia más estocástica y constante.

In [ ]:
fecha = datos_count_cuatri_mag.anyo_mes_cuatri
magnitud = datos_count_cuatri_mag.mag
cantidad = datos_count_cuatri_mag.control_ed

plt.figure(figsize=(12, 6))
color = ListedColormap(["lightseagreen", "lightgreen", "lawngreen","yellow", "gold","darkorange", "red"])

sc = plt.scatter(fecha, magnitud, c=cantidad, cmap=color, s=20, marker='o', alpha=0.7)
plt.ylabel('Magnitud (M)')
plt.title('Distribución Temporal de Magnitudes')
plt.colorbar(sc, label='Frecuencia de eventos')
plt.grid(alpha=0.3)
plt.show()

## 4. Análisis Espacial (Mapa Tectónico)
Utilizando **PyGMT**, visualizamos la distribución espacial de los epicentros. Este mapa permite identificar la geometría de la zona de subducción (Fosa Mesoamericana), donde la sismicidad se alinea con la interacción de placas.

In [ ]:
# Configuración del Mapa con PyGMT
# NOTA: Asegúrate de tener pygmt instalado correctamente.

try:
    region = [
        datos.longitude.min() - 1,
        datos.longitude.max() + 1,
        datos.latitude.min() - 1,
        datos.latitude.max() + 1,
    ]

    fig = pygmt.Figure()
    fig.basemap(region=region, projection="M15c", frame=True)
    fig.coast(land="gray", water="lightblue", shorelines=True)

    # Colorear por profundidad (cmap='geo') o magnitud
    pygmt.makecpt(cmap="jet", series=[datos.depth.min(), datos.depth.max()])

    fig.plot(
        x=datos.longitude,
        y=datos.latitude,
        size=0.05 * (2**datos.mag), # Tamaño proporcional a magnitud exponencial
        fill=datos.depth,
        cmap=True,
        style="c",
        pen="black"
    )

    fig.colorbar(frame='af+l"Profundidad (km)"')
    fig.show()
    # fig.savefig("mapa_sismicidad_chiapas.png")
except Exception as e:
    print(f"No se pudo generar el mapa PyGMT: {e}")
    print("Verifica que tengas PyGMT instalado y configurado.")

## 5. Conclusiones
1.  **Instrumentación vs. Realidad:** El análisis confirma que el aumento en la frecuencia de eventos reportados es principalmente un artefacto de la mejora tecnológica en la red de detección.
2.  **Riesgo Sísmico Latente:** A pesar de las fluctuaciones en la detección de sismos menores, la ocurrencia de sismos mayores (M > 6.0) mantiene una constancia estocástica propia de una zona de subducción activa.
3.  **Valor de los Datos:** La limpieza de datos históricos (filtrado por año y magnitud de completitud) es esencial para evitar conclusiones erróneas en estudios de amenaza sísmica.